In [1]:
from tqdm import tqdm
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util

c:\Users\cho03\Desktop\대학\4학년\2학기\캡스톤디자인-여행사이트\python\virtual\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# KoSBERT 모델 로드
model = SentenceTransformer("jhgan/ko-sbert-sts")

In [3]:
filtered_data = pd.read_csv('../../data/review_split.csv')

In [4]:
# 태그별 예시 문장 정의
tag_examples = {
    "가성비": [
        "가격 대비 만족도가 높았어요.",
        "비용 대비 기대 이상입니다.",
        "이 가격에 추천하고 싶어요.",
        "가격에 비해 훌륭한 선택이었습니다.",
        "저렴한 가격에 만족스러웠습니다.",
        "경제적인 가격에 만족했습니다.",
        "가성비가 좋았어요.",
        "가격이 정말 합리적이었어요.",
        "가격 대비 시설이 좋았습니다.",
        "가격대비 최고의 선택이었어요.",
        "이 가격에 이 정도 품질이라니 놀랐어요.",
        "합리적인 가격으로 편안하게 묵었습니다.",
        "가격이 부담되지 않아 좋았습니다.",
        "가격에 비해 시설이 기대 이상이었어요.",
        "이 가격이라면 다시 방문할 의향이 있어요."
    ],
    "청결": [
        "객실 청결 상태가 매우 잘 유지되고 있었습니다.",
        "방이 굉장히 깔끔하고 정돈된 느낌이었어요.",
        "숙소 전체가 위생적으로 관리되어서 편안하게 머물렀습니다.",
        "화장실과 침대가 모두 매우 청결하게 유지되어 있었어요.",
        "깨끗하게 관리된 공간이라서 마음 편히 지낼 수 있었습니다.",
        "숙소가 정돈이 잘 되어 있고 위생 관리가 철저해 보였습니다.",
        "전체적인 청결 상태가 아주 훌륭했어요.",
        "방 안이 쾌적하고 먼지 하나 없는 느낌이어서 좋았어요.",
        "위생 상태가 매우 잘 유지되고 있어 안심이 되었습니다.",
        "객실이 깨끗하게 관리되어 기분이 좋았습니다.",
        "침구와 바닥 모두 청결해서 불편함이 없었어요.",
        "위생적으로 관리된 숙소라서 마음이 편안했습니다.",
        "청결도가 높아 누구에게나 추천하고 싶은 숙소예요.",
        "정리 정돈이 잘 되어 있어 아주 쾌적했습니다.",
        "위생 관리가 철저히 이루어져서 신뢰가 갔습니다.",
    ],
    "서비스": [
        "직원들이 정말 친절해서 기분이 좋았습니다.",
        "사장님께서 직접 챙겨주셔서 감사했습니다.",
        "서비스가 아주 세심하고 친절했어요.",
        "프론트 직원들이 친절하게 응대해 주셨어요.",
        "직원들과 사장님 모두 친절하고 따뜻했습니다.",
        "요청 사항에 대해 직원들이 빠르게 응대해 주셨어요.",
        "사장님이 친절하게 맞아주셔서 마음이 편안했어요.",
        "직원들과 사장님 모두 서비스가 훌륭합니다.",
        "친절한 직원들과 사장님 덕분에 편안하게 머물렀습니다.",
        "직원들과 사장님이 정말 배려심 깊게 대해 주셨어요.",
        "사장님이 따뜻하게 대해주셔서 인상적이었어요.",
        "직원들이 협조적이고 사장님도 친절해서 좋았습니다.",
        "서비스가 뛰어나고 직원들이 상냥했어요.",
        "사장님과 직원들 모두 친절하게 맞아주셔서 기분 좋았습니다.",
        "요청에 빠르게 대응해 주셔서 감사했어요.",
    ],
    "위치": [
        "위치가 좋아서 근처 관광지로 이동하기 편리했어요.",
        "도심 한가운데 위치해 있어서 접근성이 뛰어났습니다.",
        "주변에 식당과 카페가 많아 편리했습니다.",
        "대중교통과 가까운 위치라 이동이 매우 편리했어요.",
        "관광 명소들과 가까워서 시간을 절약할 수 있었습니다.",
        "주요 시설과의 접근성이 매우 좋았어요.",
        "숙소의 위치가 아주 좋고 주변에 필요한 것이 모두 있었습니다.",
        "위치가 좋아 어디든 쉽게 갈 수 있었습니다.",
        "주변에 다양한 볼거리와 즐길 거리가 있어 만족스러웠어요.",
        "숙소가 조용한 곳에 위치해 있어서 편히 쉴 수 있었습니다.",
        "대중교통을 이용하기에 최적의 위치에 있습니다.",
        "도심과 가까워 모든 것이 쉽게 접근 가능했습니다.",
        "주요 관광지와 가까운 위치 덕분에 여행이 편리했어요.",
        "교통이 편리하고 중심지와 가까워 편리했습니다.",
        "편리한 위치 덕분에 모든 계획이 순조로웠어요.",
    ],
    "가족 여행": [
        "아이들과 함께 머물기에 좋은 숙소예요.",
        "가족 단위로 오기 딱 좋아요.",
        "가족과 함께 머물기에 필요한 시설이 잘 갖춰져 있어요.",
        "부모님을 모시고 오기 좋은 숙소입니다.",
        "아이들과 부모님 모두 만족할 만한 숙소였어요.",
        "부모님과 함께 이용하기 편리했습니다.",
        "가족이 함께 즐길 수 있는 숙소입니다.",
        "아이들과 부모님이 편안하게 지낼 수 있었어요.",
        "부모님과의 가족 여행에 딱 맞는 숙소입니다.",
        "아이들이 좋아할 편의시설도 잘 갖춰져 있어요.",
        "부모님과 아이들 모두 만족한 여행이었어요.",
        "가족끼리 머물기에 완벽한 공간입니다.",
        "가족 여행에 필요한 모든 것을 갖춘 숙소입니다.",
        "부모님과 아이들 모두 즐길 수 있는 숙소예요.",
        "가족이 모두 함께 편안하게 머물 수 있었습니다.",
    ],
    "연인": [
        "로맨틱한 분위기로 커플 여행에 좋습니다.",
        "남자친구와 함께 오기 딱 좋아요.",
        "특별한 날을 위해 여자친구와 방문했어요.",
        "아늑한 분위기라 남자친구와 추천합니다.",
        "연인과 오붓한 시간을 보내기 좋은 숙소입니다.",
        "여자친구와 로맨틱한 시간을 보냈어요.",
        "연인과 함께하는 특별한 장소로 추천합니다.",
        "남자친구와 함께할 아늑한 분위기입니다.",
        "기념일에 여자친구와 방문하기 딱 좋아요.",
        "남자친구와 특별한 날을 보내기 좋았어요.",
        "연인과 오붓하게 머물기 좋은 장소입니다.",
        "여자친구와 머물기에 완벽한 숙소였습니다.",
        "로맨틱한 분위기가 남자친구와 잘 맞았어요.",
        "연인과 함께하기 좋은 로맨틱한 장소입니다.",
        "커플에게 추천하는 멋진 숙소예요.",
    ],
    "풍경": [
        "자연 경관이 아름다워서 힐링되는 느낌이었습니다.",
        "숙소에서 바라보는 뷰가 정말 멋졌어요.",
        "산과 강의 경치가 인상적이고 마음이 편안해졌습니다.",
        "탁 트인 전망 덕분에 기분이 상쾌해졌어요.",
        "자연 속에서 여유를 만끽할 수 있는 숙소였어요.",
        "뷰가 너무 아름다워서 계속 바라보고 싶었습니다.",
        "숙소에서 보이는 자연 경관이 너무 좋아서 기억에 남아요.",
        "풍경이 멋져서 정말 힐링이 되었습니다.",
        "아름다운 자연과 함께하는 느낌이 들어 좋았어요.",
        "숙소에서 보는 전망이 정말 일품이었습니다.",
        "자연의 아름다움을 느끼며 여유로운 시간을 보낼 수 있었습니다.",
        "탁 트인 뷰 덕분에 아침마다 기분이 상쾌했어요.",
        "숙소 주변의 자연 경관이 너무 아름다워서 힐링이 되었습니다.",
        "창밖으로 보이는 풍경이 정말 감동적이었어요.",
        "자연 속에서 조용히 쉴 수 있는 멋진 숙소였습니다.",
    ],
}

In [5]:
# 태그 예시 문장 임베딩 생성
tag_embeddings = {}  # 빈 딕셔너리 초기화

for tag, sentences in tqdm(tag_examples.items(), desc="예시 문장 임베딩 중"):
    embeddings = model.encode(sentences, device=device)  # 문장 임베딩 생성
    tag_embeddings[tag] = embeddings  # 생성된 임베딩을 태그 키에 저장

예시 문장 임베딩 중: 100%|██████████| 7/7 [00:15<00:00,  2.23s/it]


In [6]:
# 리뷰 문장 임베딩 생성
review_embeddings = []

# 각 리뷰의 분할 문장에 대해 반복
for review_sentences in tqdm(filtered_data["review_split"], desc="리뷰 문장 임베딩 중"):
    # 문장 배치 임베딩
    embeddings = model.encode(review_sentences, device=device, show_progress_bar=False)
    review_embeddings.append(embeddings)

리뷰 문장 임베딩 중: 100%|██████████| 8792/8792 [31:26<00:00,  4.66it/s]  


In [7]:
# 태그 카운트를 위한 딕셔너리 초기화
tag_counts_list = []

# 유사도 계산 및 태그 할당
tagged_sentences = []
for review, review_embedding in tqdm(zip(filtered_data["review_split"], review_embeddings), total=len(filtered_data), desc="리뷰 유사도 계산 중"):
    # 현재 리뷰에 대한 태그 카운트 초기화
    current_tag_counts = {f"리뷰의 {tag}": 0 for tag in tag_embeddings.keys()}  # 태그 이름에 '리뷰의' 추가
    current_similarity_scores = {tag: [] for tag in tag_embeddings.keys()}  # 유사도 점수 초기화

    # 유사도 계산
    for tag, tag_embedding in tag_embeddings.items():
        # 리뷰와 태그 예시 문장 간 유사도 계산
        similarity = cosine_similarity(review_embedding.reshape(1, -1), tag_embedding)  # 유사도 계산

        # 유사도가 0.6보다 클 경우 태그 카운트 증가
        if similarity.max() >= 0.58:  # 유사도 배열에서 최대값 비교
            current_tag_counts[f"리뷰의 {tag}"] += 1  # '리뷰의'가 추가된 키로 카운트 증가

        # 유사도 점수 추가
        current_similarity_scores[tag].append(similarity.max())

    # 태그 카운트를 리스트에 추가
    tag_counts_list.append(current_tag_counts)

리뷰 유사도 계산 중: 100%|██████████| 8792/8792 [01:11<00:00, 122.12it/s]


In [8]:
# 태그 카운트 리스트를 데이터프레임으로 변환
tag_counts_df = pd.DataFrame(tag_counts_list)

# 태그 카운트를 filtered_data에 병합
filtered_data = pd.concat([filtered_data.reset_index(drop=True), tag_counts_df], axis=1)

In [9]:
# lodging_id가 포함된 filtered_data에서 태그 카운트 열만 선택
tag_columns = [f"리뷰의 {tag}" for tag in tag_embeddings.keys()]  # 태그 열 리스트 생성
tag_counts_grouped = filtered_data.groupby("lodging_id")[tag_columns].sum().reset_index()  # 숙소별 태그 카운트 집계

# 열 이름 변경
tag_counts_grouped.columns = ["lodging_id"] + [f"숙소의 {tag.replace('리뷰의 ', '')}" for tag in tag_columns]

# 원본 filtered_data와 병합
filtered_data = pd.merge(filtered_data, tag_counts_grouped, on="lodging_id", how="left")

In [10]:
filtered_data.to_csv("tag_review.csv", encoding='utf-8-sig', index=False)

In [11]:
tagging_data = filtered_data[
    [
        "lodging_id",
        "숙소의 가성비",
        "숙소의 청결",
        "숙소의 서비스",
        "숙소의 위치",
        "숙소의 가족 여행",
        "숙소의 연인",
        "숙소의 풍경",
    ]
]

# 리뷰 수 계산
review_counts = filtered_data['lodging_id'].value_counts().reset_index()
review_counts.columns = ['lodging_id', 'review_count']  # 열 이름 지정

# tagging_data에 리뷰 수 병합
tagging_data = tagging_data.merge(review_counts, on='lodging_id', how='left')

In [12]:
tagging_data = tagging_data.drop_duplicates()

In [13]:
final_tags = []

threshold_ratio = 0.3
review_count_dict = tagging_data.set_index("lodging_id")["review_count"].to_dict()

for index, row in tqdm(tag_counts_grouped.iterrows(), total=len(tag_counts_grouped), desc="최종 태그 결정 중"):
    lodging_tags = []
    lodging_id = row['lodging_id']
    
    total_reviews = review_count_dict[lodging_id]

    # Use correct tag names without "리뷰의 "
    for tag in ['숙소의 가성비', '숙소의 청결', '숙소의 서비스', '숙소의 위치', '숙소의 가족 여행', '숙소의 연인', '숙소의 풍경']:
        count = row[tag]  # Get the count for the current tag

        # Calculate the ratio
        if total_reviews > 0 and (count / total_reviews) >= threshold_ratio:
            lodging_tags.append(tag.replace('숙소의 ', ''))  # Append tag without prefix

    # Final tags concatenated or "없음"
    final_tags.append(", ".join(lodging_tags) if lodging_tags else "")

tag_counts_grouped['최종 태그'] = final_tags


최종 태그 결정 중: 100%|██████████| 422/422 [00:00<00:00, 4538.07it/s]


In [14]:
# 최종 태그를 DataFrame에 추가
tag_counts_grouped['최종 태그'] = final_tags

# 숙소의 태그 정보와 최종 태그를 포함한 DataFrame 만들기
final_tagging_data = pd.merge(tagging_data, tag_counts_grouped[['lodging_id', '최종 태그']], on='lodging_id', how='left')


In [15]:
final_tagging_data

,lodging_id,숙소의 가성비,숙소의 청결,숙소의 서비스,숙소의 위치,숙소의 가족 여행,숙소의 연인,숙소의 풍경,review_count,최종 태그
0,6,2,3,6,0,1,0,3,11,서비스
1,12,2,2,4,0,2,1,1,6,"가성비, 청결, 서비스, 가족 여행"
2,14,2,1,2,0,1,0,1,8,
3,21,0,1,1,1,0,0,4,8,풍경
4,39,3,5,4,1,1,1,0,7,"가성비, 청결, 서비스"
...,...,...,...,...,...,...,...,...,...,...
417,3740,6,11,8,1,0,0,2,32,청결
418,3751,1,4,11,0,0,0,1,152,
419,3778,1,2,2,0,0,0,0,5,"청결, 서비스"
420,3813,0,2,1,0,0,0,0,6,청결


In [16]:
final_tagging_data.to_csv('../../data/review_tags(kosbert).csv', index=False, encoding='utf-8-sig')